<a href="https://colab.research.google.com/github/shivani-202/hate-speech-detection-and-classification-/blob/main/HSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HATE SPEECH DETECTION AND CLASSIFICATION MODEL**